In [1]:
import torch
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
from torchvision import transforms, datasets, models
import torch.nn as nn

# Define the path to your dataset
path = r'D:\IIEST clg\Project\Fault Detection Panel\Dataset\Faulty_solar_panel'

# Define transformations to be applied to the images
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to a common size
    transforms.ToTensor(),  # Convert images to PyTorch tensors
])


In [2]:

# Create the ImageFolder dataset
dataset = datasets.ImageFolder(root=path, transform=transform)


In [3]:


# Create a data loader
data_loader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True)

# Access class labels
class_labels = dataset.classes
print("Class Labels:", class_labels)

# Iterate through the data loader
for images, labels in data_loader:
    # Your training/validation loop here
    print(f"Batch Shape: {images.shape}, Labels: {labels}")


Class Labels: ['Bird-drop', 'Clean', 'Dusty', 'Electrical-damage', 'Physical-Damage', 'Snow-Covered']
Batch Shape: torch.Size([32, 3, 224, 224]), Labels: tensor([4, 0, 4, 2, 1, 1, 1, 2, 2, 0, 4, 5, 5, 1, 2, 4, 3, 2, 0, 1, 1, 0, 0, 5,
        3, 2, 0, 2, 0, 4, 3, 5])
Batch Shape: torch.Size([32, 3, 224, 224]), Labels: tensor([3, 4, 1, 2, 3, 2, 1, 3, 1, 1, 2, 5, 1, 5, 0, 1, 1, 1, 2, 0, 0, 0, 5, 3,
        1, 1, 1, 0, 0, 0, 2, 2])
Batch Shape: torch.Size([32, 3, 224, 224]), Labels: tensor([0, 5, 2, 3, 2, 5, 2, 0, 0, 1, 2, 0, 0, 5, 0, 2, 0, 3, 0, 2, 0, 1, 4, 2,
        2, 1, 2, 3, 2, 1, 5, 0])
Batch Shape: torch.Size([32, 3, 224, 224]), Labels: tensor([2, 0, 2, 0, 1, 1, 0, 4, 5, 2, 1, 0, 3, 2, 3, 2, 3, 4, 3, 0, 4, 2, 1, 3,
        0, 5, 0, 5, 1, 0, 3, 3])
Batch Shape: torch.Size([32, 3, 224, 224]), Labels: tensor([0, 5, 3, 2, 1, 0, 5, 0, 1, 3, 2, 4, 2, 2, 0, 1, 3, 1, 2, 0, 4, 4, 5, 2,
        5, 4, 2, 2, 5, 1, 3, 1])
Batch Shape: torch.Size([32, 3, 224, 224]), Labels: tensor([1, 2, 2, 4, 3

In [4]:
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])


In [5]:
# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [6]:
# Define the EfficientNet model
model = models.efficientnet_b0(pretrained=True)

c:\Users\ADMIN\miniconda3\envs\pranjal\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\ADMIN\miniconda3\envs\pranjal\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [7]:
# model

In [8]:

# Adjust the final fully connected layer to match the number of classes in your dataset
model.classifier[1] = nn.Linear(1280, len(dataset.classes))
# model

In [9]:
!pip install tqdm 


In [10]:
import torch.optim as optim
from tqdm.notebook import tqdm

In [11]:


# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Move the model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Training loop
num_epochs = 5

for epoch in range(num_epochs):
    model.train()
    for images, labels in tqdm(train_loader, desc=f'Epoch {epoch + 1}/{num_epochs} - Training'):
    # for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    # Validation
    model.eval()
    with torch.no_grad():
        val_loss = 0.0
        correct = 0
        total = 0

        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            loss = criterion(outputs, labels)
            val_loss += loss.item()

        print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {val_loss / len(val_loader)}, Accuracy: {correct / total}')


Epoch 1/5 - Training:   0%|          | 0/23 [00:00<?, ?it/s]

Epoch 1/5, Loss: 1.3321632246176403, Accuracy: 0.7005649717514124


Epoch 2/5 - Training:   0%|          | 0/23 [00:00<?, ?it/s]

Epoch 2/5, Loss: 0.6275788793961207, Accuracy: 0.8305084745762712


Epoch 3/5 - Training:   0%|          | 0/23 [00:00<?, ?it/s]

Epoch 3/5, Loss: 0.6028157124916712, Accuracy: 0.847457627118644


Epoch 4/5 - Training:   0%|          | 0/23 [00:00<?, ?it/s]

Epoch 4/5, Loss: 0.49963903178771335, Accuracy: 0.8813559322033898


Epoch 5/5 - Training:   0%|          | 0/23 [00:00<?, ?it/s]

Epoch 5/5, Loss: 0.44839674482742947, Accuracy: 0.8757062146892656


In [12]:
##run that  for resnet18
## run for  densenet